#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("data/speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [4]:
docs

[Document(metadata={'source': 'data/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'data/speech.txt'}, page_content='…\n\nIt will be all the easier for us to c

In [5]:
embeddings=OllamaEmbeddings(model="nomic-embed-text:v1.5")
db=FAISS.from_documents(docs,embeddings)
db

/var/folders/pt/bx3t9ncd41x2t3vdmz_c7pvw0000gn/T/ipykernel_3380/3465169893.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model="nomic-embed-text:v1.5")


In [6]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [7]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [8]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='7e2ccfd9-5f52-4431-89a7-3802f8c6a77e', metadata={'source': 'data/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(346.26434)),
 (Document(id='4dfe2793-7f3c-40b0-9d5e-d713113121ce', metadata={'source':

In [9]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-0.3021300137042999,
 0.2790676951408386,
 -3.613821029663086,
 -0.9979861974716187,
 2.01918625831604,
 1.4695111513137817,
 -1.0231361389160156,
 0.2343270480632782,
 0.48298361897468567,
 -0.1799921840429306,
 -0.06276671588420868,
 0.5970166921615601,
 0.8667206764221191,
 1.6231967210769653,
 2.0880637168884277,
 -0.7805346250534058,
 0.2589172422885895,
 -1.2077211141586304,
 -0.6834806203842163,
 0.7262994647026062,
 -0.9002158045768738,
 -0.3933122158050537,
 0.09238004684448242,
 0.4789392054080963,
 1.3454012870788574,
 0.4108045697212219,
 -0.5247980356216431,
 0.6717443466186523,
 -1.0718083381652832,
 -0.5257762670516968,
 0.9155543446540833,
 0.06208556890487671,
 -0.05138404667377472,
 0.28213125467300415,
 -1.5352699756622314,
 -1.9181106090545654,
 0.0800088420510292,
 1.334449291229248,
 0.35603606700897217,
 -1.4277738332748413,
 0.06143443286418915,
 -0.49634385108947754,
 0.12243049591779709,
 -1.5525953769683838,
 1.366394281387329,
 -0.6893924474716187,
 -0.1293

In [10]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='7e2ccfd9-5f52-4431-89a7-3802f8c6a77e', metadata={'source': 'data/speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='4dfe2793-7f3c-40b0-9d5e-d713113121ce', metadata={'source': 'data/speech.txt'}, page_co

In [11]:
### Saving And Loading
db.save_local("faiss_index")

In [12]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [17]:
docs

[Document(page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.', metadata={'source': 'speech.txt'}),
 Document(page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the prid